# Exercises: Working with Databases in Python

We have based these exercises around this [article](https://towardsdatascience.com/do-you-know-python-has-a-built-in-database-d553989c87bd). 

Feel free to reference it as you complete the exercises below. To view it, you will need to create a FREE-tier Medium account. 

You DO NOT need a payed account to view this article.

## Import Libraries

We will need `pandas` and `sqlite3`. 

SQLite is a small, self-contained database engine that comes with Python, so you will not have to do any additional installations for this lesson. 

In the future, if you want to connect to a different database engine, such as SQL server, you may need additional libraries such as `sqlalchemy` or `pyodbc` and a tool called a 'driver' which helps to connect a database to an application. But, we won't worry about that for now.


In [3]:
# Import sqlite3 and pandas.
import sqlite3
import pandas as pd


## Create a Database

Let's say we want to create a database to store information related to books.

Use sqlite3's `connect` method to create a new database called `books.db`.

Make sure to store this connection in a variable named `con` (short for 'connection').

In [4]:
# Create the new database below
# con = sqlite3
con = sqlite3.connect("books.db")
print(con)


In [5]:
cur = con.cursor()

## Create a Table

We've created the overall `books.db`, but now we want to store some relevant information inside of it. Creating a table to track our favorite books would be a good idea!

Refer to the `python-db-walkthrough` file. There is a section where we created a table named `movies` using the sqlite3 method `executescript`.

Using this information as a reference, we want you to create a new table called `favorite_books`. 

The table should have these fields and datatypes:
1. id -> `INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT`
2. title -> a `TEXT` field
3. author -> a `TEXT` field
4. description -> a `TEXT` field 

Don't forget those `BEGIN` and `COMMIT` statements! 

In [6]:
# Create a new table in your database.

cur.executescript("""
    BEGIN;
    CREATE TABLE IF NOT EXISTS favorite_books (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        author TEXT,
        description TEXT
    );
    COMMIT;
""")

## Insert Records

Now, add 3 entries to your new table for your top 3 favorite books.

We can use the sqlite3 `executemany` method to run a sql statement for each element within a list.


Update the `data` list below to include 3 `tuples` of data, like the commented example. 
Then run the code.


In [7]:
# Add your 3 favorite books to your table.

sql = 'INSERT INTO favorite_books (title, author, description) values(?, ?, ?)'
data = [
    ('Harry Potter and the Philosopher\'s Stone', 'J.K. Rowling', 'The first book in the beloved wizarding world series'),
    ('The Hunger Games', 'Suzanne Collins', 'A dystopian novel where teens fight for survival'),
    ('Percy Jackson and the Lightning Thief', 'Rick Riordan', 'A modern take on Greek mythology featuring a young demigod'),
    ('The Golden Compass', 'Philip Pullman', 'First in the His Dark Materials trilogy, a fantasy adventure across parallel worlds'),
    ('Eragon', 'Christopher Paolini', 'A young farm boy discovers a dragon egg and becomes a Dragon Rider')
]

with con:
    con.executemany(sql, data)


## Query Your Table

Now, write a `SELECT` query using the sqlite3 `execute` method to return all the records from the `favorite_books` table.


Iterate over this result and print each row out to the console.


In [8]:

# Write your code here

cur.execute("SELECT * FROM favorite_books")
rows = cur.fetchall()

# Print each row
print("Favorite Books:")

for row in rows:
    print(f"ID: {row[0]}")
    print(f"Title: {row[1]}")
    print(f"Author: {row[2]}")
    print(f"Description: {row[3]}")
    print("-" * 30)



Favorite Books:
ID: 1
Title: Harry Potter and the Philosopher's Stone
Author: J.K. Rowling
Description: The first book in the beloved wizarding world series
------------------------------
ID: 2
Title: The Hunger Games
Author: Suzanne Collins
Description: A dystopian novel where teens fight for survival
------------------------------
ID: 3
Title: Percy Jackson and the Lightning Thief
Author: Rick Riordan
Description: A modern take on Greek mythology featuring a young demigod
------------------------------
ID: 4
Title: The Golden Compass
Author: Philip Pullman
Description: First in the His Dark Materials trilogy, a fantasy adventure across parallel worlds
------------------------------
ID: 5
Title: Eragon
Author: Christopher Paolini
Description: A young farm boy discovers a dragon egg and becomes a Dragon Rider
------------------------------
ID: 6
Title: Harry Potter and the Philosopher's Stone
Author: J.K. Rowling
Description: The first book in the beloved wizarding world series
-------

## Create a New Table with Pandas

Books are cool, but you don't have to read movies! 

Let's create a new table to store our favorite movies based on books.

This time we're going to use a `pandas` DataFrame to create the data.

Update the `title` list below to include your 3 favorite movies based on books.

Then, using pandas `to_sql` method (an example can be found in `pandas-db-walkthrough`), create a new table named `favorite_book_movies` using this Data Frame.

Finally, just like we did in the last secion, print all results of this new table to the console.

In [10]:
# Use Pandas to make a new table in your database.

df_movies = pd.DataFrame({
    'id': [1, 2, 3],
    'title': ['The Lord of the Rings: The Fellowship of the Ring', 'To Kill a Mockingbird', 'Pride and Prejudice'],
    'director': ['Peter Jackson', 'Robert Mulligan', 'Joe Wright'],
    'release_year': [2001, 1962, 2005]
})
df_movies.to_sql('favorite_book_movies', con, if_exists='replace', index=False)
with con:
    # select and print records here
 cur.execute("SELECT * FROM favorite_book_movies")
rows = cur.fetchall()

print("Favorite Book Movies:")
for row in rows:
    print(f"ID: {row[0]}")
    print(f"Title: {row[1]}")
    print(f"Director: {row[2]}")
    print(f"Release Year: {row[3]}")
    print("-" * 30)

Favorite Book Movies:
ID: 1
Title: The Lord of the Rings: The Fellowship of the Ring
Director: Peter Jackson
Release Year: 2001
------------------------------
ID: 2
Title: To Kill a Mockingbird
Director: Robert Mulligan
Release Year: 1962
------------------------------
ID: 3
Title: Pride and Prejudice
Director: Joe Wright
Release Year: 2005
------------------------------


## What's up with `with con`?

You may have noticed in the `python-db-walkthrough` file that we had to include a section at the end to `close()` our sqlite3 connection. 

If we didn't include this code, we would end up with a hanging open connection, which could lead to database locking and/or record corruption. Both of these are very bad things!

But, it would be hard to remember to always directly close the connection when we need to.

That's why we have the `with` keyword! You can read about this Python language feature [here](https://www.geeksforgeeks.org/with-statement-in-python/).

In this context, `with con` will open up a connection to the database and then close it once the Python code inside the indented block finishes. 

This way, we don't have to manually close our db connections in our code and we have a nice visual block of code to help manage each operation performed against the database.